In [ ]:
# for similarity need to download larger english word library with the below in gitbash 
# python -m spacy download en_core_web_lg

In [1]:
import numpy as np
import pandas as pd
import spacy
import pyspark
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

In [3]:
file = pd.read_csv("final_moviedata.csv")
file_all = pd.read_csv("final_moviedata.csv")

In [4]:
file = pd.DataFrame(file)
file.head()

,imdb_id,title,original_title,overview,poster_link
0,tt0069049,The Other Side of the Wind,The Other Side of the Wind,"Orson Welles' unfinished masterpiece, restored...",https://m.media-amazon.com/images/M/MV5BNDg5NT...
1,tt3906082,Mary Shelley,Mary Shelley,The love affair between poet Percy Shelley and...,https://m.media-amazon.com/images/M/MV5BNzUyM2...
2,tt5613402,Mobile Homes,Mobile Homes,"In forgotten towns along the American border, ...",https://m.media-amazon.com/images/M/MV5BMTc3OT...
3,tt3038708,Iron Sky: The Coming Race,Iron Sky: The Coming Race,"Twenty years after the events of Iron Sky, the...",https://m.media-amazon.com/images/M/MV5BYzFmZD...
4,tt3498786,Sly Cooper,Sly Cooper,Sly Cooper is a kinetic and comedic heist film...,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...


In [5]:
file_all = pd.DataFrame(file_all)
file_all.head()

,imdb_id,title,original_title,overview,poster_link
0,tt0069049,The Other Side of the Wind,The Other Side of the Wind,"Orson Welles' unfinished masterpiece, restored...",https://m.media-amazon.com/images/M/MV5BNDg5NT...
1,tt3906082,Mary Shelley,Mary Shelley,The love affair between poet Percy Shelley and...,https://m.media-amazon.com/images/M/MV5BNzUyM2...
2,tt5613402,Mobile Homes,Mobile Homes,"In forgotten towns along the American border, ...",https://m.media-amazon.com/images/M/MV5BMTc3OT...
3,tt3038708,Iron Sky: The Coming Race,Iron Sky: The Coming Race,"Twenty years after the events of Iron Sky, the...",https://m.media-amazon.com/images/M/MV5BYzFmZD...
4,tt3498786,Sly Cooper,Sly Cooper,Sly Cooper is a kinetic and comedic heist film...,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...


In [6]:
file_all.count()

imdb_id           1886
title             1886
original_title    1886
overview          1886
poster_link       1886
dtype: int64

In [7]:
file.drop_duplicates(keep='first', inplace=True)
file_all.drop_duplicates(keep='first', inplace=True)

In [8]:
file.count()

imdb_id           1886
title             1886
original_title    1886
overview          1886
poster_link       1886
dtype: int64

In [29]:
overview_df2 = file[["imdb_id","title", "poster_link", "overview"]].copy()
overview_df2.dropna(inplace=True)
overview_df2 = overview_df2.iloc[87:189]
overview_df2.head()

,imdb_id,title,poster_link,overview
87,tt3371366,Transformers: The Last Knight,https://m.media-amazon.com/images/M/MV5BMTk3OT...,"Autobots and Decepticons are at war, with huma..."
88,tt6194530,Ava,https://m.media-amazon.com/images/M/MV5BM2UyNz...,"Ava, 13, is spending the summer on the Atlanti..."
89,tt7044010,Rory Scovel Tries Stand-Up for the First Time,https://m.media-amazon.com/images/M/MV5BZWUxM2...,Comedian Rory Scovel storms the stage in Atlan...
90,tt5539054,You Get Me,https://m.media-amazon.com/images/M/MV5BMjI5OD...,"After arguing with his girlfriend, Ali, Tyler ..."
91,tt4572792,The Book of Henry,https://m.media-amazon.com/images/M/MV5BMjQ3Mj...,"Naomi Watts stars as Susan, a single mother of..."


In [10]:
df_all = file_all[["imdb_id","title", "poster_link", "overview"]].copy()
df_all.dropna(inplace=True)
df_all.head()

,imdb_id,title,poster_link,overview
0,tt0069049,The Other Side of the Wind,https://m.media-amazon.com/images/M/MV5BNDg5NT...,"Orson Welles' unfinished masterpiece, restored..."
1,tt3906082,Mary Shelley,https://m.media-amazon.com/images/M/MV5BNzUyM2...,The love affair between poet Percy Shelley and...
2,tt5613402,Mobile Homes,https://m.media-amazon.com/images/M/MV5BMTc3OT...,"In forgotten towns along the American border, ..."
3,tt3038708,Iron Sky: The Coming Race,https://m.media-amazon.com/images/M/MV5BYzFmZD...,"Twenty years after the events of Iron Sky, the..."
4,tt3498786,Sly Cooper,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...,Sly Cooper is a kinetic and comedic heist film...


In [11]:
overview_df2.isna().any()

imdb_id        False
title          False
poster_link    False
overview       False
dtype: bool

In [12]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [13]:
sentenceData = spark.createDataFrame(overview_df2.astype(str))
sentenceData.show() 

+---------+--------------------+--------------------+--------------------+
|  imdb_id|               title|         poster_link|            overview|
+---------+--------------------+--------------------+--------------------+
|tt0069049|The Other Side of...|https://m.media-a...|Orson Welles' unf...|
|tt3906082|        Mary Shelley|https://m.media-a...|The love affair b...|
|tt5613402|        Mobile Homes|https://m.media-a...|In forgotten town...|
|tt3038708|Iron Sky: The Com...|https://m.media-a...|Twenty years afte...|
|tt3498786|          Sly Cooper|http://ia.media-i...|Sly Cooper is a k...|
|tt2328678| The King's Daughter|https://m.media-a...|King Louis XIV's ...|
|tt3859310|How to Talk to Gi...|https://m.media-a...|A couple of Briti...|
|tt4765284|     Pitch Perfect 3|https://m.media-a...|Sequel to Pitch P...|
|tt5690244|            Machines|https://m.media-a...|This portrayal of...|
|tt2582498|      Sweet Virginia|https://m.media-a...|A former rodeo ch...|
|tt0974015|      Justice 

In [14]:
sentenceData_all = spark.createDataFrame(df_all.astype(str))
sentenceData_all.show() 

+---------+--------------------+--------------------+--------------------+
|  imdb_id|               title|         poster_link|            overview|
+---------+--------------------+--------------------+--------------------+
|tt0069049|The Other Side of...|https://m.media-a...|Orson Welles' unf...|
|tt3906082|        Mary Shelley|https://m.media-a...|The love affair b...|
|tt5613402|        Mobile Homes|https://m.media-a...|In forgotten town...|
|tt3038708|Iron Sky: The Com...|https://m.media-a...|Twenty years afte...|
|tt3498786|          Sly Cooper|http://ia.media-i...|Sly Cooper is a k...|
|tt2328678| The King's Daughter|https://m.media-a...|King Louis XIV's ...|
|tt3859310|How to Talk to Gi...|https://m.media-a...|A couple of Briti...|
|tt4765284|     Pitch Perfect 3|https://m.media-a...|Sequel to Pitch P...|
|tt5690244|            Machines|https://m.media-a...|This portrayal of...|
|tt2582498|      Sweet Virginia|https://m.media-a...|A former rodeo ch...|
|tt0974015|      Justice 

In [15]:
tokenizer = Tokenizer(inputCol="overview", outputCol="words")
tokenizer

Tokenizer_f005f9dae1c7

In [16]:
# Transform and show DataFrame
tokenized = tokenizer.transform(sentenceData)
tokenized_all = tokenizer.transform(sentenceData_all)
tokenized.show(truncate=False)

+---------+-------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
# Instantiate Remover
remover = StopWordsRemover(inputCol="words", outputCol="nostopwords")

In [18]:
# Transform and show data
tokenized = remover.transform(tokenized)
tokenized_all = remover.transform(tokenized_all)
tokenized.show(truncate=True)

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  imdb_id|               title|         poster_link|            overview|               words|         nostopwords|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|tt0069049|The Other Side of...|https://m.media-a...|Orson Welles' unf...|[orson, welles', ...|[orson, welles', ...|
|tt3906082|        Mary Shelley|https://m.media-a...|The love affair b...|[the, love, affai...|[love, affair, po...|
|tt5613402|        Mobile Homes|https://m.media-a...|In forgotten town...|[in, forgotten, t...|[forgotten, towns...|
|tt3038708|Iron Sky: The Com...|https://m.media-a...|Twenty years afte...|[twenty, years, a...|[twenty, years, e...|
|tt3498786|          Sly Cooper|http://ia.media-i...|Sly Cooper is a k...|[sly, cooper, is,...|[sly, cooper, kin...|
|tt2328678| The King's Daughter|https://m.media-a...|King Louis 

In [19]:
tokenized_all.show(truncate=True)

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  imdb_id|               title|         poster_link|            overview|               words|         nostopwords|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|tt0069049|The Other Side of...|https://m.media-a...|Orson Welles' unf...|[orson, welles', ...|[orson, welles', ...|
|tt3906082|        Mary Shelley|https://m.media-a...|The love affair b...|[the, love, affai...|[love, affair, po...|
|tt5613402|        Mobile Homes|https://m.media-a...|In forgotten town...|[in, forgotten, t...|[forgotten, towns...|
|tt3038708|Iron Sky: The Com...|https://m.media-a...|Twenty years afte...|[twenty, years, a...|[twenty, years, e...|
|tt3498786|          Sly Cooper|http://ia.media-i...|Sly Cooper is a k...|[sly, cooper, is,...|[sly, cooper, kin...|
|tt2328678| The King's Daughter|https://m.media-a...|King Louis 

# Starting Spacy Similarity code

In [20]:
# Load English tokenizer, tagger, parser, NER and word vectors
#MAKE SURE THIS IS LARGE LIBRARY

nlp = spacy.load("en_core_web_lg")



## adding columns for similarities

In [21]:
from pyspark.sql.functions import lit

In [22]:
#Code to compare each movie to the all movie and recommend top 3
ll = []
result = []
for m in tokenized.collect():
    main_row = str(m.nostopwords)
    docA = nlp(main_row)
    for f in tokenized_all.collect():

        overview_row = str(f.nostopwords)
        #print(overview_row)
        doc_row = nlp(overview_row)

        compare = docA.similarity(doc_row)
        ll.append({'imdb_id': f.imdb_id, 'Title': f.title,'Poster': f.poster_link,
           'Similarity': compare })
        # sample = tokenized.withColumn('similarity', lit(str(ll)))
        # print(ll)   
        df_simi = pd.DataFrame(ll)
        df_simi.sort_values(by=['Similarity'], inplace=True, ascending=False)
        df_simi.reset_index(drop=True)
        top3_df = df_simi.iloc[1:4]
        top3_df = top3_df[["imdb_id","Title", "Poster", "Similarity"]]
#         top3_df.head()
    result.append({'imdb_id': m.imdb_id,'Title': m.title,'Poster': m.poster_link, 
        'Recom1': top3_df.iloc[0,0], 'Recom1Poster': top3_df.iloc[0,2],
        'Recom2': top3_df.iloc[1,0], 'Recom2Poster': top3_df.iloc[1,2],
        'Recom3': top3_df.iloc[2,0],'Recom3Poster': top3_df.iloc[2,2]})
    print(result)


[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg', 'Recom2': 'tt5687040', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg', 'Recom3': 'tt4177972', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'}]
[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._

[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg', 'Recom2': 'tt5687040', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg', 'Recom3': 'tt4177972', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'}, {'imdb_id': 'tt3906082', 'Title': 'Mary Shelley', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQXVyODY3Nzc0OTk@._V1_SX300.jpg', 'Recom1': 'tt3906082', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQ

[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg', 'Recom2': 'tt5687040', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg', 'Recom3': 'tt4177972', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'}, {'imdb_id': 'tt3906082', 'Title': 'Mary Shelley', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQXVyODY3Nzc0OTk@._V1_SX300.jpg', 'Recom1': 'tt3906082', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQ

[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg', 'Recom2': 'tt5687040', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg', 'Recom3': 'tt4177972', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'}, {'imdb_id': 'tt3906082', 'Title': 'Mary Shelley', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQXVyODY3Nzc0OTk@._V1_SX300.jpg', 'Recom1': 'tt3906082', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQ

[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg', 'Recom2': 'tt5687040', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg', 'Recom3': 'tt4177972', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'}, {'imdb_id': 'tt3906082', 'Title': 'Mary Shelley', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQXVyODY3Nzc0OTk@._V1_SX300.jpg', 'Recom1': 'tt3906082', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQ

...

[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg', 'Recom2': 'tt5687040', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg', 'Recom3': 'tt4177972', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'}, {'imdb_id': 'tt3906082', 'Title': 'Mary Shelley', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQXVyODY3Nzc0OTk@._V1_SX300.jpg', 'Recom1': 'tt3906082', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQ

[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg', 'Recom2': 'tt5687040', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg', 'Recom3': 'tt4177972', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'}, {'imdb_id': 'tt3906082', 'Title': 'Mary Shelley', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQXVyODY3Nzc0OTk@._V1_SX300.jpg', 'Recom1': 'tt3906082', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQ

[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg', 'Recom2': 'tt5687040', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg', 'Recom3': 'tt4177972', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'}, {'imdb_id': 'tt3906082', 'Title': 'Mary Shelley', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQXVyODY3Nzc0OTk@._V1_SX300.jpg', 'Recom1': 'tt3906082', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQ

[{'imdb_id': 'tt0069049', 'Title': 'The Other Side of the Wind', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg', 'Recom1': 'tt5925968', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg', 'Recom2': 'tt5687040', 'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg', 'Recom3': 'tt4177972', 'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'}, {'imdb_id': 'tt3906082', 'Title': 'Mary Shelley', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQXVyODY3Nzc0OTk@._V1_SX300.jpg', 'Recom1': 'tt3906082', 'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQ

KeyboardInterrupt: 

In [24]:
result

[{'imdb_id': 'tt0069049',
  'Title': 'The Other Side of the Wind',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNDg5NTQ0NDU1Nl5BMl5BanBnXkFtZTgwMTExNzIyNjM@._V1_SX300.jpg',
  'Recom1': 'tt5925968',
  'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BOGQzYjg0ZGItNjdhNS00NTJmLTgxOTUtMjEyMTM4Y2VmMTlkXkEyXkFqcGdeQXVyNTExMTg3MDg@._V1_SX300.jpg',
  'Recom2': 'tt5687040',
  'Recom2Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxMTQ5NzAzOV5BMl5BanBnXkFtZTgwNzI2MzY4NDM@._V1_SX300.jpg',
  'Recom3': 'tt4177972',
  'Recom3Poster': 'https://m.media-amazon.com/images/M/MV5BOTM3YTY3YmQtZmM2Yy00MDgzLWIwYzAtZjkyMGUzNWRjNWJjXkEyXkFqcGdeQXVyNTE0MzM3MDc@._V1_SX300.jpg'},
 {'imdb_id': 'tt3906082',
  'Title': 'Mary Shelley',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYxYzMyZDgyNGUyXkEyXkFqcGdeQXVyODY3Nzc0OTk@._V1_SX300.jpg',
  'Recom1': 'tt3906082',
  'Recom1Poster': 'https://m.media-amazon.com/images/M/MV5BNzUyM2EzOTEtOTMwMi00NzMwLWEyYTAtZGYx

# Importing recommandation to csv

In [25]:
df_recomm = pd.DataFrame(result)
df_recomm.drop_duplicates(keep='first', inplace=True)
df_recomm.head()

,Poster,Recom1,Recom1Poster,Recom2,Recom2Poster,Recom3,Recom3Poster,Title,imdb_id
0,https://m.media-amazon.com/images/M/MV5BNDg5NT...,tt5925968,https://m.media-amazon.com/images/M/MV5BOGQzYj...,tt5687040,https://m.media-amazon.com/images/M/MV5BMTkxMT...,tt4177972,https://m.media-amazon.com/images/M/MV5BOTM3YT...,The Other Side of the Wind,tt0069049
1,https://m.media-amazon.com/images/M/MV5BNzUyM2...,tt3906082,https://m.media-amazon.com/images/M/MV5BNzUyM2...,tt5925968,https://m.media-amazon.com/images/M/MV5BOGQzYj...,tt5687040,https://m.media-amazon.com/images/M/MV5BMTkxMT...,Mary Shelley,tt3906082
2,https://m.media-amazon.com/images/M/MV5BMTc3OT...,tt5613402,https://m.media-amazon.com/images/M/MV5BMTc3OT...,tt3906082,https://m.media-amazon.com/images/M/MV5BNzUyM2...,tt5131906,https://m.media-amazon.com/images/M/MV5BNzMxOD...,Mobile Homes,tt5613402
3,https://m.media-amazon.com/images/M/MV5BYzFmZD...,tt5613402,https://m.media-amazon.com/images/M/MV5BMTc3OT...,tt3038708,https://m.media-amazon.com/images/M/MV5BYzFmZD...,tt3906082,https://m.media-amazon.com/images/M/MV5BNzUyM2...,Iron Sky: The Coming Race,tt3038708
4,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...,tt3498786,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...,tt5613402,https://m.media-amazon.com/images/M/MV5BMTc3OT...,tt3906082,https://m.media-amazon.com/images/M/MV5BNzUyM2...,Sly Cooper,tt3498786


In [26]:
#Reorganize the column
df_recomm = df_recomm[["imdb_id","Title", "Poster", 
        "Recom1", "Recom1Poster", "Recom2", "Recom2Poster",
                      "Recom3", "Recom3Poster"]]
df_recomm.head()

,imdb_id,Title,Poster,Recom1,Recom1Poster,Recom2,Recom2Poster,Recom3,Recom3Poster
0,tt0069049,The Other Side of the Wind,https://m.media-amazon.com/images/M/MV5BNDg5NT...,tt5925968,https://m.media-amazon.com/images/M/MV5BOGQzYj...,tt5687040,https://m.media-amazon.com/images/M/MV5BMTkxMT...,tt4177972,https://m.media-amazon.com/images/M/MV5BOTM3YT...
1,tt3906082,Mary Shelley,https://m.media-amazon.com/images/M/MV5BNzUyM2...,tt3906082,https://m.media-amazon.com/images/M/MV5BNzUyM2...,tt5925968,https://m.media-amazon.com/images/M/MV5BOGQzYj...,tt5687040,https://m.media-amazon.com/images/M/MV5BMTkxMT...
2,tt5613402,Mobile Homes,https://m.media-amazon.com/images/M/MV5BMTc3OT...,tt5613402,https://m.media-amazon.com/images/M/MV5BMTc3OT...,tt3906082,https://m.media-amazon.com/images/M/MV5BNzUyM2...,tt5131906,https://m.media-amazon.com/images/M/MV5BNzMxOD...
3,tt3038708,Iron Sky: The Coming Race,https://m.media-amazon.com/images/M/MV5BYzFmZD...,tt5613402,https://m.media-amazon.com/images/M/MV5BMTc3OT...,tt3038708,https://m.media-amazon.com/images/M/MV5BYzFmZD...,tt3906082,https://m.media-amazon.com/images/M/MV5BNzUyM2...
4,tt3498786,Sly Cooper,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...,tt3498786,http://ia.media-imdb.com/images/M/MV5BNmEwNjJm...,tt5613402,https://m.media-amazon.com/images/M/MV5BMTc3OT...,tt3906082,https://m.media-amazon.com/images/M/MV5BNzUyM2...


In [27]:
#Use this code for the first time to create the csv file
df_recomm.to_csv('/Users/ericH/Desktop/moviesDashboard/raw/recom.csv', 
                 index = None, header=True)


In [ ]:
#Use this code to add to csv file
# df_recomm.to_csv('/Users/ericH/Desktop/moviesDashboard/raw/recom.csv', mode='a', index = None, header=False)


In [ ]:
file_path = '/Users/ericH/Desktop/moviesDashboard/raw/recom.csv'
with open(file_path, mode='a', newline='\n') as f:
            df_recomm.to_csv(f, index = None, header=False)